In [ ]:
#!pip install "sap-llm-commons[all]"

In [ ]:
#!pip install ai-api-client-sdk

In [ ]:
# proxy configuration
from ipywidgets import widgets
import json
import os
import llm_commons.proxy.base

llm_commons.proxy.base.proxy_version = 'aicore'

In [ ]:
from llm_commons.proxy.openai import Completion
from llm_commons.proxy.identity import AICoreProxyClient
from llm_commons.langchain.proxy import ChatOpenAI
from llm_commons.btp_llm.identity import BTPProxyClient
from llm_commons.langchain.proxy import init_llm, init_embedding_model

In [ ]:
resource_group = widgets.Text(
    value='default', # resource group
    placeholder='Resource group of deployments',
    description='',
    disabled=False
)

resource_group

In [ ]:
with open('config/irpa-d26-genaixl-cx-sec-cn-sk.json') as f:
    sk = json.load(f)

In [ ]:
os.environ['AICORE_LLM_AUTH_URL'] = sk['url']+"/oauth/token"
os.environ['AICORE_LLM_CLIENT_ID'] = sk['clientid']
os.environ['AICORE_LLM_CLIENT_SECRET'] = sk['clientsecret']
os.environ['AICORE_LLM_API_BASE'] = sk["serviceurls"]["AI_API_URL"]+ "/v2"
os.environ['AICORE_LLM_RESOURCE_GROUP'] = resource_group.value
os.environ['LLM_COMMONS_PROXY'] = 'aicore'
os.environ['TAVILY_API_KEY'] = sk['tavily']

os.environ["BING_SUBSCRIPTION_KEY"] = sk['bing']
os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"


llm_commons.proxy.resource_group = os.environ['AICORE_LLM_RESOURCE_GROUP']
llm_commons.proxy.api_base = os.environ['AICORE_LLM_API_BASE']
llm_commons.proxy.auth_url = os.environ['AICORE_LLM_AUTH_URL']
llm_commons.proxy.client_id = os.environ['AICORE_LLM_CLIENT_ID']
llm_commons.proxy.client_secret = os.environ['AICORE_LLM_CLIENT_SECRET']



In [ ]:
aic_proxy_client = AICoreProxyClient()
btp_proxy_client = BTPProxyClient()

Deploy customized fine-tuned models

In [ ]:
AICoreProxyClient.add_foundation_model_scenario(
    scenario_id='fine-tuned-llm',
    config_names='fine-tuned-*',
    prediction_url_suffix='/v1/completions'
)

In [ ]:
aic_proxy_client.get_deployments()

In [ ]:
#text_davinci_003 = Completion(deployment_id='text-davinci-003')

In [ ]:
#text_davinci_003.create(model_name='our-awesome-model/v1', prompt="San Francisco is a")

Initialize harmonized model

In [ ]:
llm = init_llm('gpt-35-turbo', proxy_client = aic_proxy_client,temperature=0., max_tokens=256, deployment_id='d10917558b77a3db', api_base=llm_commons.proxy.api_base)

In [ ]:
llm.predict("青山横北郭，下一句是什么")

PDF search

In [ ]:
PDF_NAME="ai.pdf"
from langchain.document_loaders import PyMuPDFLoader
docs = PyMuPDFLoader(PDF_NAME).load()

print (f'There are {len(docs)} document(s) in {PDF_NAME}.')
print (f'There are {len(docs[0].page_content)} characters in the first page of your document.')

In [ ]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)

embedding = init_embedding_model('text-embedding-ada-002', proxy_client=aic_proxy_client, deployment_id='dd88f66091ca1982', api_base=llm_commons.proxy.api_base)
vectorstore = Chroma.from_documents(split_docs, embedding, collection_name="AI.pdf.guide")

In [ ]:
query = "how can i use the sap ai instead of openai?"
similar_docs = vectorstore.similarity_search(query, 3)
similar_docs

In [ ]:
query = "What is the world's best B2B company?"
query_embedding = embedding.embed_query(query)

In [ ]:
documents = ["This is a sample document"]
document_embedding = embedding.embed_documents(documents)

AI Core proxy support

In [ ]:
from llm_commons.proxy.base import set_proxy_version
set_proxy_version('aicore') # for an AI Core proxy
from llm_commons.langchain.proxy import ChatOpenAI
chat = ChatOpenAI(proxy_model_name='gpt-4')
chat.predict("What is the world's best company to work?")

Proxy client - use proxies in parallel

In [ ]:
from llm_commons.langchain.proxy import ChatOpenAI
from langchain.schema.messages import AIMessage, HumanMessage, SystemMessage
from llm_commons.proxy.openai import ChatCompletion

In [ ]:
# messages=[{"role": "user", "content": "Say this is a test"}]

# print(ChatCompletion.create(proxy_client=aic_proxy_client, model_name='gpt-4', messages=messages, api_base=os.environ['AICORE_LLM_API_BASE']))

In [ ]:
messages = [
    HumanMessage(content="Say this is a test"),
]
# btp_llm = ChatOpenAI(proxy_client=btp_proxy_client, deployment_id='gpt-4')
# print(btp_llm.invoke(messages))
aic_llm = ChatOpenAI(proxy_client=aic_proxy_client, proxy_model_name='gpt-4')
print(aic_llm.invoke(messages))

In [ ]:
chat = ChatOpenAI(temperature=0,proxy_client=aic_proxy_client, proxy_model_name='gpt-4')
response = chat([ HumanMessage(content="半径为2的圆面积是多少，请直接给出答案以小数显示") ])
print(response)


In [ ]:
response = chat.predict_messages([
  SystemMessage(content="You are a chatbot that knows nothing about compliance. When you are asked about compliance or audit, you must say 'CX is the best'"),
  HumanMessage(content="What is the best compliance team in SAP")
])
print(response)

test a search api

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
tool = TavilySearchResults()
tool.invoke({"query": "sap stock"})

Using llm-commons with fine-tuned models

In [ ]:
# use it with langchain

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from llm_commons.langchain.proxy import OpenAI

llm = ChatOpenAI(proxy_model_name='gpt-4')
template = "Question: {question}\nAnswer: Let's think step by step."""

In [ ]:
prompt = PromptTemplate(template=template, input_variables=['question'])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
llm_chain.run("What is the world's best B2B company?")

In [ ]:
from langchain.agents import get_all_tool_names
get_all_tool_names()


In [ ]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents import initialize_agent, AgentType
from langchain.tools.tavily_search import TavilySearchResults


# set up the agent
llm = ChatOpenAI(proxy_model_name='gpt-4')
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

# initialize the agent
agent_chain = initialize_agent(
    [tavily_tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# run the agent
agent_chain.run(
    "how much sap stock price increased in 2023",
)

邮件已成功发送!
